### Loading data and libraries

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from pprint import pprint
from scipy.stats import uniform
import statsmodels.api as sm
import math
import shap 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, validation_curve
from sklearn.pipeline import Pipeline as Pipeline, make_pipeline as make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.svm import SVC, LinearSVC
from sklearn.utils import resample
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler, SMOTENC
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.pipeline import Pipeline as Imbpipeline
from imblearn.pipeline import make_pipeline as Imb_make_pipeline
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel

# the first dataset includes authors with at least 1 article to form cognitive proximity.
# df_data_set1 = pd.read_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_data_set_North_America_1a.csv')

# the second dataset includes authors with at least 2 article to form cognitive proximity.
df_data_set = pd.read_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_data_set_North_America_2a.csv')


In [3]:
df_descriptive = df_data_set[['collaboration_binary', 'Geo_Dist', 'TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']]
df_descriptive.describe()

,collaboration_binary,Geo_Dist,TENB,Cog_Dist,Prov_Border,Country_Border,NotContig
count,45938.000000,45938.000000,45938.000000,45938.000000,45938.000000,45938.000000,45938.000000
mean,0.007249,1921.257450,0.034830,0.868790,0.938286,0.158061,0.969023
std,0.084832,1279.389092,0.531146,0.393080,0.240637,0.364802,0.173256
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,803.890948,0.000000,0.585058,1.000000,0.000000,1.000000
50%,0.000000,1641.795564,0.000000,0.957755,1.000000,0.000000,1.000000
75%,0.000000,3045.886033,0.000000,1.185170,1.000000,0.000000,1.000000
max,1.000000,6383.414016,20.000000,1.805018,1.000000,1.000000,1.000000


In [ ]:
print(len(set(df_data_set2.Author_1.unique().tolist())))
print(df_data_set2[df_data_set2.number_of_collaborations>0].number_of_collaborations.sum())

In [4]:
# Pearson correlation

df_descriptive = df_data_set[['collaboration_binary', 'Geo_Dist', 'TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']]
df_descriptive.corr()

from scipy.stats import pearsonr
rho = df_descriptive.corr()
pval = df_descriptive.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

,collaboration_binary,Geo_Dist,TENB,Cog_Dist,Prov_Border,Country_Border,NotContig
collaboration_binary,1.0***,-0.09***,0.64***,-0.17***,-0.23***,-0.03***,0.01**
Geo_Dist,-0.09***,1.0***,-0.07***,0.05***,0.34***,0.06***,0.19***
TENB,0.64***,-0.07***,1.0***,-0.14***,-0.19***,-0.03***,0.01**
Cog_Dist,-0.17***,0.05***,-0.14***,1.0***,0.04***,-0.02***,0.02***
Prov_Border,-0.23***,0.34***,-0.19***,0.04***,1.0***,0.11***,-0.05***
Country_Border,-0.03***,0.06***,-0.03***,-0.02***,0.11***,1.0***,-0.21***
NotContig,0.01**,0.19***,0.01**,0.02***,-0.05***,-0.21***,1.0***


### Regression analysis

In [12]:
columns = [  'Log_Geo_Dist',
             'Log_TENB', 
             'Cog_Dist',
             'Prov_Border',
             'Country_Border',
             'NotContig',
             'Log_Geo_Dist X Log_TENB', 
             'Log_Geo_Dist_Sq X Log_TENB']


Ind_v = df_data_set[columns]
Dep_v = df_data_set.collaboration_binary

columns_reg1 = [ 'Log_Geo_Dist',
                 'Log_TENB',
                 'Cog_Dist',
                 'Prov_Border',
                 'Country_Border',
                 'NotContig']

columns_reg2 = [ 'Log_Geo_Dist',
                 'Log_Geo_Dist X Log_TENB', 
                 'Cog_Dist',
                 'Prov_Border',
                 'Country_Border',
                 'NotContig']


columns_reg3 = [ 'Log_Geo_Dist',
                 'Log_Geo_Dist X Log_TENB',
                 'Log_Geo_Dist_Sq X Log_TENB',
                 'Cog_Dist',
                 'Prov_Border',
                 'Country_Border',
                 'NotContig']

X_reg1 = Ind_v[columns_reg1]
X_reg2 = Ind_v[columns_reg2]
X_reg3 = Ind_v[columns_reg3]


X_reg_1 = sm.tools.tools.add_constant(X_reg1, prepend=True, has_constant='add')
X_reg_2 = sm.tools.tools.add_constant(X_reg2, prepend=True, has_constant='add')
X_reg_3 = sm.tools.tools.add_constant(X_reg3, prepend=True, has_constant='add')


models = [X_reg1, X_reg2, X_reg3]

for model in models:
    logit_model=sm.Logit(Dep_v, model)
    result=logit_model.fit(method_kwargs={"warn_convergence": False})
    print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.013486
         Iterations 12
                           Results: Logit
Model:              Logit                Pseudo R-squared: 0.686    
Dependent Variable: collaboration_binary AIC:              1251.0244
Date:               2021-11-01 19:57     BIC:              1303.4347
No. Observations:   45938                Log-Likelihood:   -619.51  
Df Model:           5                    LL-Null:          -1972.4  
Df Residuals:       45932                LLR p-value:      0.0000   
Converged:          1.0000               Scale:            1.0000   
No. Iterations:     12.0000                                         
---------------------------------------------------------------------
                 Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
---------------------------------------------------------------------
Log_Geo_Dist    -0.5068    0.0667   -7.6033  0.0000  -0.6374  -0.3761
Log_TENB         2.6824

In [ ]:
# the geo distance that maximize the TENB's Elasticity

np.exp(1.5718/(2*0.0995))

In [ ]:
1.5718 * math.log1p(2693.219136978802) - 0.0995 *  math.log1p(2693.219136978802) *  math.log1p(2693.219136978802)

In [ ]:
dis = np.linspace (0,8691, num = 8691)

X_dis = []
for i in dis:
    x_dis = math.log1p(i)
    X_dis.append(x_dis)
    
Y_tenb = []
Y_tenb_ = []

for j in X_dis:
    y_tenb_ = 1.5718 * j - 0.0995 * j * j
    Y_tenb_.append(y_tenb_)

plt.figure(figsize=(20,10))
plt.plot (dis, Y_tenb_, color = 'red')
plt.xlabel('Physical distance (km)', size = 20)
plt.ylabel('Elasticity of TENB (network proximity)', size = 20)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)

### Machine Learning

In [ ]:
features = ['Log_Geo_Dist','Log_TENB', 'Cog_Dist', 'Prov_Border', 'Country_Border', 'NotContig']

X = df_data_set[features]
y = df_data_set.collaboration_binary

In [ ]:
unique, count = np.unique (y, return_counts = True)

y_value_count = {k : v for (k,v) in zip(unique,count)}

print ('Dataset', y_value_count)
print("\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y)

unique_train, count_train = np.unique (y_train, return_counts = True)

y_train_value_count = {k : v for (k,v) in zip(unique_train,count_train)}

print ('Dataset Train', y_train_value_count)
print("\n")

unique_test, count_test = np.unique (y_test, return_counts = True)

y_test_value_count = {k : v for (k,v) in zip(unique_test,count_test)}

print ('Dataset Test', y_test_value_count)
print("\n")


In [ ]:
logreg = LogisticRegression(max_iter = 500)
gnb = GaussianNB()
knn = KNeighborsClassifier(n_jobs = -1)
rnd = RandomForestClassifier(random_state = 123, n_jobs = -1)
xgb = XGBClassifier(use_label_encoder=False, eval_metric = 'logloss', random_state = 123, n_jobs = -1)
cat = CatBoostClassifier(random_state = 123, verbose = False)
# params = {'iterations':1000,
#         'learning_rate':0.01,
#         'depth':3,
#         'eval_metric':'F1',
#         'loss_function': "Logloss",
#         'verbose':False,
#         'od_type':"Iter", # overfit detector
#         'od_wait':500, # most recent best iteration to wait before stopping
#         'random_state' : 123}

# cat1 = CatBoostClassifier(**params)
# logreg = LogisticRegression(max_iter = 500)
# rnd = RandomForestClassifier(random_state = 123, n_jobs = -1)
# gbc = GradientBoostingClassifier(random_state = 123)
# xgb = XGBClassifier(use_label_encoder=False, eval_metric = 'logloss', random_state = 123, n_jobs = -1)
# cat = CatBoostClassifier()
# knn = KNeighborsClassifier(n_jobs = -1)
# gnb = GaussianNB()
# rus = RandomUnderSampler()
# ros = RandomOverSampler()
# smt = SMOTENC(categorical_features = [3,4,5,6], random_state = 123, n_jobs = -1)
# smtk = SMOTETomek(n_jobs = 6)
# smnn = SMOTEENN(n_jobs = 6)
# scl = StandardScaler()
# feature_selection_selbest = SelectKBest(chi2, k=7)
# feature_selection_selmodel = SelectFromModel(rnd)
# voth = VotingClassifier(estimators=[('LogisticRegression', logreg), ('RandomForestClassifier', rnd), ('GradientBoostingClassifier', gbc), ('XGBClassifier', xgb), ('KNeighborsClassifier', knn),('GaussianNB', gnb)], voting='hard')
# vots = VotingClassifier(estimators=[('LogisticRegression', logreg), ('RandomForestClassifier', rnd), ('GradientBoostingClassifier', gbc), ('XGBClassifier', xgb), ('KNeighborsClassifier', knn),('GaussianNB', gnb)], voting='soft')
# voth = VotingClassifier(estimators=[('RandomForestClassifier', rnd), ('XGBClassifier', xgb)], voting='hard')
# vots = VotingClassifier(estimators=[('RandomForestClassifier', rnd), ('XGBClassifier', xgb)], voting='soft')

In [ ]:
# Cross Validation

skf = StratifiedKFold(n_splits=5, shuffle = True)

for clf, label in zip([logreg, gnb, knn, rnd, xgb, cat], ['LogisticRegression', 'GaussianNB', 'KNeighborsClassifier', 'Random Forest', 'XGB Classifier', 'CatBoostClassifier']):
    scores = cross_val_score(clf, X_train, y_train, scoring='f1', cv=skf, n_jobs=-1)
    print("F1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    

In [ ]:
# Test scores
for clf, label in zip([logreg, gnb, knn, rnd, xgb, cat], ['LogisticRegression', 'GaussianNB', 'KNeighborsClassifier', 'Random Forest', 'XGB Classifier', 'CatBoostClassifier']):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
#     print("Presicion: %0.3f [%s]" % (precision_score(y_test, y_pred), label))
#     print("Recall: %0.3f [%s]" % (recall_score(y_test, y_pred), label))
    print("F1: %0.2f [%s]" % (f1_score(y_test, y_pred), label))
#     print("Accuracy: %0.2f [%s]" % (accuracy_score(y_test, y_pred), label))
#     print('\n')

In [ ]:
# baseline (RandomForest)

rus = RandomUnderSampler()
ros = RandomOverSampler()
smt = SMOTENC(categorical_features = [3,4,5], random_state = 123, n_jobs = -1)
scl = StandardScaler()

pipe_scl = Pipeline(steps = [['SCL', scl], ['RandomForestClassifier', xgb]])
pipe_smt = Imbpipeline(steps = [['SMOTENC', smt], ['RandomForestClassifier', xgb]])
pipe_ros = Imbpipeline(steps = [['RandomOverSampler', ros], ['RandomForestClassifier', xgb]])
pipe_rus = Imbpipeline(steps = [['RandomUnderSampler', rus], ['RandomForestClassifier', xgb]])

for clf, label in zip([xgb, pipe_scl, pipe_smt, pipe_ros, pipe_rus], ['No preprocess', 'SCL', 'SMOTENC', 'RandomOverSampler', 'RandomUnderSampler']):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
#     print("Presicion: %0.3f [%s]" % (precision_score(y_test, y_pred), label))
#     print("Recall: %0.3f [%s]" % (recall_score(y_test, y_pred), label))
    print("F1: %0.2f [%s]" % (f1_score(y_test, y_pred), label))
#     print("Accuracy: %0.3f [%s]" % (accuracy_score(y_test, y_pred), label))
#     print('\n')


In [ ]:
# Feature importance - mean decrease in impurity

importances = xgb.feature_importances_ * 100


df_importance = pd.DataFrame()

df_importance.insert(0,'Feature','')
df_importance.insert(1,'importance',0)

j = 0
for i in features:
    df_importance.loc[j,'Feature'] = i
    df_importance.loc[j,'importance'] = importances[j]
    j += 1

df_importance.sort_values(by=['importance'], ascending=False, inplace = True)

df_importance[:10].plot(x = 'Feature', y = 'importance', kind = 'bar', rot = 90, ylabel = 'mean decrease in impurity (%)')

In [ ]:
# Feature importance - SHAP

xgb_explainer = shap.TreeExplainer(xgb)
shap_values = xgb_explainer.shap_values(X_train, y_train)
shap_values2 = xgb_explainer(X_train)
shap_interaction_values = xgb_explainer.shap_interaction_values(X_train)


In [ ]:
shap.plots.bar(shap_values2)
shap.summary_plot(shap_values2, X_train, plot_type="bar")
shap.summary_plot(shap_values2, X_train, plot_size=2, cmap=plt.get_cmap("cool"))

# shap.plots.waterfall(shap_values2[0])
# shap.plots.heatmap(shap_values2)

In [ ]:
# Dependence plot

shap.dependence_plot('Log_TENB', shap_values, X_train, interaction_index=None, show = False)

plt.gcf().set_size_inches(10, 10)

shap.dependence_plot(('Log_TENB', 'Log_Geo_Dist'), shap_interaction_values, X_train, show = False, cmap=plt.get_cmap("hsv"), dot_size = 70)

plt.gcf().set_size_inches(10, 10)
# ax = plt.axes()
# ax.set_facecolor("black")

# shap.dependence_plot('Cog_Dist', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('Country_Border', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('Prov_Border', shap_values, X_train, interaction_index=None)
# shap.dependence_plot('NotContig', shap_values, X_train, interaction_index=None)


In [ ]:
# Hyperparameter tuning - Randomforestclassifier - RandomSearch


n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 26)]

criterion = ['gini', 'entropy']

max_features = ['auto', 'sqrt', .1, .2, .3, .4, .5]

max_depth = [int(x) for x in np.linspace(40, 60, num = 20)]
max_depth.append(None)

min_samples_split = [2,3,4,5,6,7,8,9]

min_samples_leaf = [1,2]

bootstrap = [True, False]

param_dist = { 'n_estimators': n_estimators,
               'criterion': criterion,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# print('Parameters used in the grid:\n')
# pprint(param_dist)

skf = StratifiedKFold(n_splits = 5)

rnd_srch_model = RandomizedSearchCV(estimator = rnd,
                           param_distributions = param_dist, 
                           cv = skf, n_jobs = -1, 
                           verbose = 1,
                           n_iter = 200,
                           scoring = 'f1',
                           random_state = 123)

rnd_srch_model.fit(X_train, y_train)
# y_pred = rnd_srch_model.best_estimator_.predict(X_test)
print ('Best Parameters', rnd_srch_model.best_params_)
print ('Best Score', rnd_srch_model.best_score_)

# print ('Presicion:' , precision_score(y_test, y_pred))
# print ('Recall:' , recall_score(y_test, y_pred))
# print ('F1_Score:', f1_score(y_test, y_pred))


# Best Parameters {'n_estimators': 170, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 55, 'criterion': 'gini', 'bootstrap': True}
# Best Score 0.8852592313348533

In [ ]:
# Random Search Best parameters result (test data)

rnd_best = RandomForestClassifier(random_state = 123, n_jobs = -1,
                                       n_estimators = 170,
                                       min_samples_split = 5,
                                       min_samples_leaf = 1,
                                       max_features = 'sqrt',
                                       criterion = 'gini',
                                       max_depth = 55,
                                       bootstrap = True)

rnd_best.fit(X_train, y_train)
y_pred_best = rnd_best.predict(X_test)
print ('F1_Score:', f1_score(y_test, y_pred_best))
print ('Presicion:' , precision_score(y_test, y_pred_best))
print ('Recall:' , recall_score(y_test, y_pred_best))
print ('Accuracy:', accuracy_score(y_test, y_pred_best))
print ('confusion_matrix:')
print (confusion_matrix(y_test, y_pred_best))
print ('Classification Report:')
print (classification_report(y_test, y_pred_best))

# rfc_cv_score = cross_val_score(rnd_best, X_train, y_train, cv = skf, scoring = 'f1')

# print ('F1_Score_cv_score:', rfc_cv_score.mean())


In [ ]:
# Hyperparameter tuning - Randomforestclassifier - Grid search


n_estimators = [int(x) for x in np.linspace(start = 160, stop = 180, num = 3)]

max_features = ['auto','sqrt']

max_depth = [int(x) for x in np.linspace(53, 57, num = 5)]
max_depth.append(None)

min_samples_split = [4,5,6]

min_samples_leaf = [1]

bootstrap = [True, False]

criterion = ['gini', 'entropy']


param_grid = { 'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion' : criterion}

print('Parameters used in the grid:\n')
pprint(param_grid)

skf = StratifiedKFold(n_splits = 5)

grid_model = GridSearchCV(estimator = rnd,
                         param_grid = param_grid, 
                         cv = skf, n_jobs = -1, 
                         verbose = 1,
                         scoring = 'f1')

grid_model.fit(X_train, y_train)

print ('Best Parameters', grid_model.best_params_)
print ('Best Score', grid_model.best_score_)


# Best Parameters {'bootstrap': True, 'criterion': 'gini', 'max_depth': 53, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 170}
# Best Score 0.8852592313348533

In [ ]:
# grid Search Best parameters result (test data)

grd_best = RandomForestClassifier(random_state = 123, n_jobs = -1,
                                       criterion = 'gini',
                                       n_estimators = 170,
                                       min_samples_split = 5,
                                       min_samples_leaf = 1,
                                       max_features = 'auto',
                                       max_depth = 53,
                                       bootstrap = True)

grd_best.fit(X_train, y_train)
y_pred_best_grd = grd_best.predict(X_test)
print ('F1_Score:', f1_score(y_test, y_pred_best_grd))
print ('Presicion:' , precision_score(y_test, y_pred_best_grd))
print ('Recall:' , recall_score(y_test, y_pred_best_grd))
print ('Accuracy:', accuracy_score(y_test, y_pred_best_grd))
print ('roc_auc_score:', roc_auc_score(y_test, y_pred_best_grd))
print ('confusion_matrix:')
print (confusion_matrix(y_test, y_pred_best_grd))
print ('Classification Report:')
print (classification_report(y_test, y_pred_best_grd))

# rfc_cv_score = cross_val_score(rnd_best, X_train, y_train, cv = skf, scoring = 'f1')

# print ('F1_Score_cv_score:', rfc_cv_score.mean())
